In [2]:
! pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.2 MB 33.0 MB/s 
     |████████████████████████████████| 164 kB 87.1 MB/s 
     |████████████████████████████████| 182 kB 98.4 MB/s 
     |████████████████████████████████| 237 kB 93.0 MB/s 
     |████████████████████████████████| 78 kB 8.1 MB/s 
     |████████████████████████████████| 4.7 MB 86.4 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 51 kB 8.5 MB/s 
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=4209fcf8252a556984a777fcae656b3d16ee1100ff86a6189c1a8dd657a3ab34
  Stored in directory: /root/.cache/pip/wheels/5f/55/ab/36a76989f7f88d9ca7b1f68da6d94252bb6a8d6ad4f18e04e9
Successfully built validators


In [5]:

import streamlit as st
import numpy as np
import functions
from PIL import Image

# App Title
st.title("Pneumodetector APP")

# Introduction text
st.markdown(unsafe_allow_html=True, body="<p>Welcome to Pneumodetector APP.</p>"
                                         "<p>This is a basic app built with Streamlit."
                                         "With this app, you can upload a Chest X-Ray image and predict if the patient "
                                         "from that image suffers pneumonia or not.</p>"
                                         "<p>The model used is a Convolutional Neural Network (CNN) and in this "
                                         "moment has a test accuracy of "
                                         "<strong>90.7%.</strong></p>")

st.markdown("First, let's load an X-Ray Chest image.")

# Loading model

# Img uploader
img = st.file_uploader(label="Load X-Ray Chest image", type=['jpeg', 'jpg', 'png'], key="xray")

if img is not None:
    # Preprocessing Image
    p_img = functions.preprocess_image(img)

    if st.checkbox('Zoom image'):
        image = np.array(Image.open(img))
        st.image(image, use_column_width=True)
    else:
        st.image(p_img)

    # Loading model
    loading_msg = st.empty()
    loading_msg.text("Predicting...")
    model = functions.load_model()

    # Predicting result
    prob, prediction = functions.predict(model, p_img)

    loading_msg.text('')

    if prediction:
        st.markdown(unsafe_allow_html=True, body="<span style='color:red; font-size: 50px'><strong><h4>Pneumonia! :slightly_frowning_face:</h4></strong></span>")
    else:
        st.markdown(unsafe_allow_html=True, body="<span style='color:green; font-size: 50px'><strong><h3>Healthy! :smile: </h3></strong></span>")

    st.text(f"*Probability of pneumonia is {round(prob[0][0] * 100, 2)}%")

  command:

    streamlit run /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py [ARGUMENTS]
2022-10-22 17:23:41.955 
  command:

    streamlit run /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py [ARGUMENTS]


In [4]:
import keras
import numpy as np
import streamlit as st
from keras import layers, models, optimizers  # modeling
from PIL import Image

MODEL = "cnn_augmented_dropout_90_7.h5"


@st.cache(allow_output_mutation=True)
def load_model():
    print("loading model")
    model = keras.models.load_model(f"model/{MODEL}", compile=True)

    return model


def preprocess_image(img):
    image = Image.open(img).convert("RGB")
    p_img = image.resize((224, 224))

    return np.array(p_img) / 255.0


def predict(model, img):
    prob = model.predict(np.reshape(img, [1, 224, 224, 3]))

    if prob > 0.5:
        prediction = True
    else:
        prediction = False

    return prob, prediction